In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain_groq import ChatGroq

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
llm = ChatGroq(model="openai/gpt-oss-120b",
api_key=os.getenv("GROQ_API_KEY"),
temperature=0.5)

In [6]:
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001E49A15C2F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E49A15CD70>, model_name='openai/gpt-oss-120b', temperature=0.5, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
import PyPDF2


In [20]:
import langchain
from langchain_core.globals import set_debug

set_debug(True)

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
RESPONSE_JSON_STRING = json.dumps(RESPONSE_JSON, indent=2)

In [11]:
quiz_generation_prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """You are an expert MCQ maker. 
        It is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
        
        Make sure the questions are not repeated and check all the questions to be conforming the text as well.
        Make sure to format your response like the RESPONSE_JSON below and use it as a guide. 
        Ensure to make {number} MCQs.
        
        ### RESPONSE_JSON
        {response_json}"""
    ),
    (
        "human", 
        "Text: {text}"
    )
])

In [26]:
quiz_chain = quiz_generation_prompt | llm | StrOutputParser()

In [21]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# --- PROMPT 2: EVALUATION (Your new template) ---
review_prompt = ChatPromptTemplate.from_template(
    """You are an expert english grammarian and writer. 
    Given a Multiple Choice Quiz for {subject} students.
    You need to evaluate the complexity of the question and give a complete analysis of the quiz.
    
    Quiz_MCQs:
    {quiz}
    
    Check from an expert English Writer of the above quiz:"""
)

In [23]:
# Input: {subject, quiz} -> Output: String (The Review)
review_chain = review_prompt | llm | StrOutputParser()

In [27]:
generate_evaluate_chain = (
    # Step 1: Pass inputs through, but ALSO run quiz_chain and store result in 'quiz'
    RunnablePassthrough.assign(quiz=quiz_chain)
    
    # Step 2: Now that 'quiz' is in the state, run review_chain and store in 'review'
    | RunnablePassthrough.assign(review=review_chain)
)

In [28]:
file_path = r"C:\Users\Loba\automcqgen\data.txt"
file_path

'C:\\Users\\Loba\\automcqgen\\data.txt'

In [29]:
with open(file_path, "r") as file:
    TEXT = file.read()